# Classification Rules and their applications
### Part I. Implementation

Here I present my implementation of the Rule Learner algorithm. While it is probably possible to further improve performance, I could not find a way to do so.

## 1. PRISM algorithm

## 1.1. First: two classes

### `Condition` class with printing.

In [1]:
class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            if self.true_false:
                return "{}>={}".format(self.attribute, self.value)
            else:
                return "{}<{}".format(self.attribute, self.value)

### `Rule` class with printing.

In [2]:
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class

    def add_condition(self, condition):
        self.conditions.append(condition)

    def set_params(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage



    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)

## 1.2. Constructing filters

Given a condition or a list of conditions, these functions construct a filtering condition to be applied to the current pandas data frame. Both functions return a string. The string then can be converted into a boolean condition using `eval`. This method was proposed by Colin Pinney. 

In [3]:
def condition_filter(condition, dataset_name):
    if condition is None:
        return ""
    if condition.true_false is None:  # categorical attribute
        return '('+dataset_name+'["' + condition.attribute + '"]' + "==" + '"' + condition.value + '")'
    if condition.true_false: # >= numeric value
        return '('+dataset_name+'["' + condition.attribute + '"]' + ">=" + str(condition.value) + ")"
    return '('+dataset_name+'["' + condition.attribute + '"]' + "<" + str(condition.value) + ")"


def condition_list_filter(condition_list, dataset_name):
    result = ""

    for cond in condition_list:
        result += condition_filter(cond, dataset_name) + " & "

    result += "True"
    return result

## 1.3. Best condition

This function tries all different combinations of attribute-values and seeks the combination that would give the best accuracy of the rule, where accuracy is defined as number of records which have both condition and class over all records with the current condition.

The function takes as a parameter a current subset of data, and performs only reading operations on this subset - tocount the accuracy. It also evaluates a coverage of the rule which is defined as a total number of records which satisfy current condition.

The function returns an object representing the condition with best accuracy. If the coverage falls below the specified threshold, the function returns None. 

In [4]:
def get_best_condition(columns, current_subset, prev_conditions, 
                       class_label, min_coverage=30, prev_best_accuracy=0.0):
    
    # we do not check the same column that was already used to generate this subset
    used_attributes = [x.attribute for x in prev_conditions]
    
    best_accuracy = prev_best_accuracy
    best_coverage = None
    best_col = None
    best_val = None
    best_true_false = None

    # we iterate over all attributes except the class - which is in the last column
    for col in columns[:-1]:
        # we do not use the same column in one rule
        if col in used_attributes:
            continue

        # Extract unique values from the column
        unique_vals = current_subset[col].unique().tolist()

        # Consider each unique value in turn
        # The treatment is different for numeric and categorical attributes
        for val in unique_vals:
            if isinstance(val, int) or isinstance(val, float):
                # Here we construct 2 conditions:
                # if actual value >= val or if actual value < val

                # First if actual value >= val                
                new_condition = Condition(col, val, True)

                # create a filtering condition
                filter = condition_filter(new_condition, "current_subset")

                # total covered by current condition
                total_covered = len(current_subset[eval(filter)])
                if total_covered >= min_coverage:
                    # total with this condition and a given class
                    total_correct = len(current_subset[(current_subset[columns[-1]] == class_label) & eval(filter)])

                    acc = total_correct/total_covered
                    if acc > best_accuracy or (acc == best_accuracy and
                                               (best_coverage is None or total_covered > best_coverage)):
                        best_accuracy = acc
                        best_coverage = total_covered
                        best_col = col
                        best_val = val
                        best_true_false = True

                # now repeat the same for the case - if actual value < val                
                new_condition = Condition(col, val, False)

                # create a filtering condition
                filter = condition_filter(new_condition, "current_subset")

                # total covered by current condition
                total_covered = len(current_subset[eval(filter)])
                if total_covered >= min_coverage:
                    # total with this condition and a given class
                    total_correct = len(current_subset[(current_subset[columns[-1]] == class_label) & eval(filter)])

                    acc = total_correct / total_covered
                    if acc > best_accuracy or (acc == best_accuracy and
                                               (best_coverage is None or total_covered > best_coverage)):
                        best_accuracy = acc
                        best_coverage = total_covered
                        best_col = col
                        best_val = val
                        best_true_false = False

            else: # categorical attribute
                # For categorical attributes - this is just single condition if actual value == val
                new_condition = Condition(col, val)

                # create a filtering condition
                filter = condition_filter(new_condition, "current_subset")

                # total covered by current condition
                total_covered = len(current_subset[eval(filter)])

                if total_covered >= min_coverage:
                    # total with this condition and a given class
                    total_correct = len(current_subset[(current_subset[columns[-1]] == class_label) & eval(filter)])

                    acc = total_correct / total_covered
                    if acc > best_accuracy or (acc == best_accuracy and
                                               (best_coverage is None or total_covered > best_coverage)):
                        best_accuracy = acc
                        best_coverage = total_covered
                        best_col = col
                        best_val = val
                        best_true_false = None

    if best_col is None:
        return None
    return Condition(best_col,best_val, best_true_false)

## 1.4. Learn one rule

Here we try to learn a single rule. We start with a single best condition, and try adding more conditions to improve the accuracy as long as the coverage does not fall below the specified threshold.

In [5]:
def learn_one_rule(columns, current_data, class_label,
                   min_coverage=30):

    covered_subset = None
    
    # start with creating a new Rule with a single best condition
    current_rule = Rule(class_label)
    best_condition = get_best_condition(columns, current_data, [], class_label, min_coverage)

    if best_condition is None:
        return None

    current_rule.add_condition(best_condition)
    
    # create a filtering condition
    filter = condition_filter (best_condition,"current_data")

    # total covered by current condition
    total_covered = len(current_data[eval(filter)])
    
    if total_covered < min_coverage:
        return None

    # total with this condition and a given class
    total_correct = len(current_data[(current_data[columns[-1]] == class_label) & eval(filter)])

    current_accuracy = total_correct / total_covered
    current_rule.set_params(current_accuracy, total_covered )  

    if current_accuracy == 1.0:
        return current_rule

    # leave only a subset where the best condition holds
    covered_subset = current_data[eval(filter)]

    # repeatedly try to improve Rule's accuracy as long as coverage remains sufficient
    while True:
        best_condition = get_best_condition(columns, covered_subset, current_rule.conditions,
                                            class_label, min_coverage, current_accuracy)

        if best_condition is None: 
            return current_rule

        # create an additional filtering condition on the current subset
        filter = condition_filter(best_condition, "covered_subset")

        # total covered by current condition
        total_covered = len(covered_subset[eval(filter)])

        if total_covered < min_coverage:
            return current_rule  # we could not improve previous rule

        # total with this condition and a given class
        total_correct = len(covered_subset[(covered_subset[columns[-1]] == class_label) & eval(filter)])

        new_accuracy = total_correct / total_covered

        current_rule.add_condition(best_condition)
        current_rule.set_params(new_accuracy, total_covered)
        current_accuracy = new_accuracy

        if new_accuracy == 1: # stop improving
            return current_rule

        # update subset to continue working with
        covered_subset = covered_subset[eval(filter)]

## 1.5. Learn all rules

This implementation follows the logic of the original PRISM algorithm. It processes each class in turn. 
Because high-accuracy rules generated by the algorithm are disjoint with respect to the class label, this is not a problem when we are just interested in dicovering some pieces of knowledge - not in constructing the classification table.
For the Decision Table the order in which the rules are discovered matters, and we should process all classes at the same time, as shown in the lecture examples. 

In [6]:
def learn_rules(columns, data, classes=None,
                min_coverage=30, min_accuracy=0.6):
    # List of final rules
    rules = []

    # If list of classes of interest is not provided - it is extracted from the last column of data
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()

    # one class label at a time
    for class_label in class_labels:
        done = False
        while len(current_data) >= min_coverage and not done:
            # Learn a rule with a single condition
            rule = learn_one_rule(columns, current_data, class_label, min_coverage)

            # The best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break

            # If we get the rule with coverage above threshold
            # We check if it passes accuracy threshold
            if rule.accuracy >= min_accuracy:
                rules.append(rule)
                             
                # create a filtering condition
                filter = condition_list_filter(rule.conditions,"current_data")
                
                # drop all the rows where all rule conditions hold
                current_data = current_data.drop(current_data[eval(filter)].index)
            else:
                done = True

    return rules

# 2. Correctness test

We will test the algorithm on a small Contact Lenses dataset from the original paper. 

The dataset was downloaded from [here](https://archive.ics.uci.edu/ml/datasets/Lenses). The CSV version is included in this repository.

**Attribute Information**:

3 Classes:
- __1__ : the patient should be fitted with __hard__ contact lenses,
- __2__ : the patient should be fitted with __soft__ contact lenses,
- __3__ : the patient should __not__ be fitted with contact lenses.


Attributes:
1. age of the patient: (1) young, (2) pre-presbyopic, (3) presbyopic
2. spectacle prescription:  (1) myope, (2) hypermetrope
3. astigmatic:     (1) no, (2) yes
4. tear production rate:  (1) reduced, (2) normal

Presbyopia is physiological insufficiency of accommodation associated with the aging of the eye that results in progressively worsening ability to focus clearly on close objects. So "age=presbiopic" means old.

Hypermetropia: far-sightedness, also known as long-sightedness - cannot see close.
Myopia: nearsightedness - cannot see at distance.

In [7]:
import pandas as pd
import numpy as np

data_file = "contact_lenses.csv"
data = pd.read_csv(data_file, index_col=['id'])
data.columns

Index(['age', 'spectacles', 'astigmatism', 'tear production rate',
       'lenses type'],
      dtype='object')

We can replace numbers with actual values - for clarity of presentation.

In [8]:
# classes
conditions = [ data['lenses type'].eq(1), data['lenses type'].eq(2), data['lenses type'].eq(3)]
choices = ["hard","soft","none"]

data['lenses type'] = np.select(conditions, choices)

# age groups
conditions = [ data['age'].eq(1), data['age'].eq(2), data['age'].eq(3)]
choices = ["young","medium","old"]

data['age'] = np.select(conditions, choices)

# spectacles
conditions = [ data['spectacles'].eq(1), data['spectacles'].eq(2)]
choices = ["nearsighted","farsighted"]

data['spectacles'] = np.select(conditions, choices)

# astigmatism
conditions = [ data['astigmatism'].eq(1), data['astigmatism'].eq(2)]
choices = ["no","yes"]

data['astigmatism'] = np.select(conditions, choices)

# tear production rate
conditions = [ data['tear production rate'].eq(1), data['tear production rate'].eq(2)]
choices = ["reduced","normal"]

data['tear production rate'] = np.select(conditions, choices)

We run the algorithm next:

In [9]:
column_list = data.columns.to_numpy().tolist()
rules = learn_rules(column_list, data, None, 1, 0.95)

We print all the rules in the order they were discovered. Note that first all the rules for class "None" were discovered, then for class "Soft", and finally for class "Hard".

In [10]:
for rule in rules:
    print(rule)

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [age=old, tear production rate=normal, spectacles=nearsighted, astigmatism=no] then none. Coverage:1, accuracy: 1.0
If [spectacles=farsighted, astigmatism=yes, age=medium] then none. Coverage:1, accuracy: 1.0
If [age=old, spectacles=farsighted, astigmatism=yes] then none. Coverage:1, accuracy: 1.0
If [astigmatism=no] then soft. Coverage:5, accuracy: 1.0
If [astigmatism=yes] then hard. Coverage:4, accuracy: 1.0


These rules precisely correspond to the rules computed by hand according to the algorithms's logic. The full step-by-step example is in [this file](https://docs.google.com/spreadsheets/d/1dw4X_veScVo0x1AS9wHTI9vxTfoemKoj/edit?usp=sharing&ouid=106298942908841514891&rtpof=true&sd=true).

We want however to see rules sorted by accuracy, and for the same accuracy - sorted by coverage.

In [11]:
from operator import attrgetter

# sort rules by accuracy descending
rules.sort(key=attrgetter('accuracy', 'coverage'), reverse=True)
for rule in rules:
    print(rule)

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [astigmatism=no] then soft. Coverage:5, accuracy: 1.0
If [astigmatism=yes] then hard. Coverage:4, accuracy: 1.0
If [age=old, tear production rate=normal, spectacles=nearsighted, astigmatism=no] then none. Coverage:1, accuracy: 1.0
If [spectacles=farsighted, astigmatism=yes, age=medium] then none. Coverage:1, accuracy: 1.0
If [age=old, spectacles=farsighted, astigmatism=yes] then none. Coverage:1, accuracy: 1.0


Finally we want to see what rules would be discovered if the best condition is determined using all the classes. 
The modified code is in [rule_learner_both_classes.py](rule_learner_both_classes.py).

In [12]:
import rule_learner_both_classes as rlbc

In [13]:
rules = rlbc.learn_rules(column_list, data, None, 1, 0.95)

In [14]:
from operator import attrgetter

# sort rules by accuracy descending
rules.sort(key=attrgetter('accuracy', 'coverage'), reverse=True)
for rule in rules:
    print(rule)

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0
If [astigmatism=yes, spectacles=nearsighted] then hard. Coverage:3, accuracy: 1.0
If [age=old] then none. Coverage:2, accuracy: 1.0
If [spectacles=nearsighted] then soft. Coverage:2, accuracy: 1.0
If [age=medium] then none. Coverage:1, accuracy: 1.0
If [age=young] then hard. Coverage:1, accuracy: 1.0


This also precisely corresponds to the step-by-step example. So it seems that our implementation is correct.

Copyright &copy; 2022 Marina Barsky. All rights reserved.